In [1]:
import multiprocessing as mp
import sys
#add the path were the models are
sys.path.append("../inProduction/")
import pandas as pd
import numpy as np
from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format
from itertools import repeat
import time
from functools import reduce
from sirPSO import SIR_PSO
#set default theme for plts
theme_set(theme_linedraw())

In [2]:
data = pd.read_csv("../data/estados.csv")
#Select only Sao Paulo
sp = data[data["state"] == "SP"]
#Remove missing values to not crash the intervals
sp = sp.dropna()
sp.head()

,date,state,newCases,mortes,TOTAL,totalCasesPred,sucetivel,Recuperado
41,2020-02-25,SP,1.0,0.0,1.0,1.000000,4.591905e+07,0.000000
42,2020-02-26,SP,0.0,0.0,1.0,1.253639,4.591905e+07,0.140394
43,2020-02-27,SP,0.0,0.0,1.0,1.571138,4.591905e+07,0.316136
44,2020-02-28,SP,0.0,0.0,1.0,1.968710,4.591905e+07,0.536200
45,2020-02-29,SP,1.0,0.0,2.0,2.468231,4.591905e+07,0.812695


In [3]:
#create a series with the cummulative number of cases
y = np.array(sp["TOTAL"])

#Give the number of days since the day of first case confirmed
x = range(0,len(sp["TOTAL"]))

In [4]:
#start model
model = SIR_PSO(tamanhoPop = 50000000, numeroProcessadores = 8)

In [5]:
def bootstratpTS(npArray, replicate):
    simList = []
    def poissonGen(npArray, replicate = None):
        simSeries = []
        for i in range(0,len(npArray)):
            if i == 0:
                simSeries.append(npArray[i]) 
            else:
                simSeries.append(np.random.poisson(lam = npArray[i] - npArray[i-1], size = 1)[0])
        return np.cumsum(np.array(simSeries))
    for i in range(0,replicate):
        simList.append(poissonGen(npArray))
    return np.array(simList)
    
def runSir(model, y, x, ndays):
    newx = range(0, len(x) + ndays) 
    model.fit(y = y, x = x)
    return model.predict(newx)
    
def predictCI(model, y, x, start, ndays, bootstrap, n_jobs):
    """
    This function fits diffent models to data to get confidence interval for I + R.
    y = an array with the series of cases
    x = an range object with the first and last day of cases
    start =  a date in format "YYYY-mm-dd" indicating the day of the first case reported
    ndays = number of days to be predicted
    bootstrap = number of times that the model will run
    n_jobs = number of core to be used to fit the models
        
    """
    model = model
    #Create a lol with data for run the model
    lists = bootstratpTS(npArray = y, replicate = bootstrap)
        
    #Model will be fitted and predicted so R) using ci is not consisent
    #Make cores avalible to the process
    pool =  mp.Pool(processes = n_jobs)
        
    #Run the model
    results = pool.starmap(runSir, [(model, lists[i], x, ndays) for i in range(0,len(lists))])
        
    #Create data frames for models
    pred = [results[i] for i in range(0,len(results))]

        
    return pred

In [6]:
model = SIR_PSO(tamanhoPop = 50000000)

In [7]:
predictCI(model = model, y = y, x = x, start = None, ndays = 7, bootstrap = 5, n_jobs = 1)

2020-04-17 07:48:08,259 - pyswarms.single.global_best - INFO - Optimize for 500 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|500/500, best_cost=8.38e+3
2020-04-17 07:48:25,161 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 8376.971304206087, best pos: [0.40295636 0.19937598]
2020-04-17 07:48:25,164 - pyswarms.single.global_best - INFO - Optimize for 500 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|500/500, best_cost=9.68e+3
2020-04-17 07:48:44,736 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 9682.732485656386, best pos: [0.4036456  0.19989463]
2020-04-17 07:48:44,740 - pyswarms.single.global_best - INFO - Optimize for 500 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|500/500, best_cost=9.6e+3
2020-04-17 07:49:04,027 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 9599.28530223

[array([1.00000000e+00, 1.44970276e+00, 2.00093848e+00, 2.67682756e+00,
        3.50820064e+00, 4.55780701e+00, 5.84365816e+00, 7.39053898e+00,
        9.33957224e+00, 1.17130577e+01, 1.45913001e+01, 1.82161503e+01,
        2.26040056e+01, 2.79809540e+01, 3.47206075e+01, 4.28293999e+01,
        5.28913965e+01, 6.54066888e+01, 8.04204316e+01, 9.92342407e+01,
        1.22446866e+02, 1.50277466e+02, 1.84842140e+02, 2.26968180e+02,
        2.79283714e+02, 3.42836013e+02, 4.21131655e+02, 5.17131814e+02,
        6.34706035e+02, 7.78983619e+02, 9.56190528e+02, 1.17327291e+03,
        1.43875479e+03, 1.76325150e+03, 2.16189240e+03, 2.65060650e+03,
        3.25011438e+03, 3.98468934e+03, 4.88505868e+03, 5.98858729e+03,
        7.34154559e+03]),
 array([1.00000000e+00, 1.45051654e+00, 2.00284420e+00, 2.68018822e+00,
        3.51349511e+00, 4.56572635e+00, 5.85495385e+00, 7.40627121e+00,
        9.36123845e+00, 1.17420096e+01, 1.46302550e+01, 1.82679811e+01,
        2.26711412e+01, 2.80702813e+01

In [ ]:
model.fit(x = x, y = y)
model.predict(x)

In [ ]:
from recombinator.optimal_block_length import optimal_block_length as opl
opl(y)

In [ ]:
from recombinator.block_bootstrap import circular_block_bootstrap as cbb

In [ ]:
arrays = cbb(x = y, block_length = 8, replications = 1000, replace = True)

In [ ]:
sp["lb"] = np.cumsum(np.quantile(np.transpose(arrays), q  = 0.0275, axis = 1))
sp["ub"] = np.cumsum(np.quantile(np.transpose(arrays), q  = 0.975, axis = 1))
sp["meanSeries"] = np.cumsum(np.median(np.transpose(arrays), axis = 1))

sp["date"] = pd.to_datetime(sp["date"])

In [ ]:
#plot graph using ggplot
(ggplot(sp) + 
    geom_line(aes(x = "date", y = "newCases")) +
    geom_line(aes(x = "date", y = "meanSeries")) +
    geom_ribbon(aes(x = "date", ymin = "lb", ymax = "ub"), alpha = 0.5) +
    scale_x_datetime(breaks = date_breaks('5 days'), labels=date_format('%d-%b')) +
    ggtitle("Series of covid-19 for Sao Paulo") +
    ylab("Number of cases"))

In [ ]:
def bootstratpTS(npArray, replicate):
    """
    
    """
    simList = []
    def poissonGen(npArray, replicate = None):
        simSeries = []
        for i in range(0,len(npArray)):
            if i == 0:
                simSeries.append(npArray[i]) 
            else:
                simSeries.append(np.random.poisson(lam = npArray[i] - npArray[i-1], size = 1)[0])
        return np.cumsum(np.array(simSeries))
    for i in range(0,replicate):
        simList.append(poissonGen(npArray))
    return np.array(simList)
simList = bootstratpTS(npArray = y, replicate = 1000)

In [ ]:
sp["plb"] = np.quantile(np.transpose(simList), q  = 0.0275, axis = 1)
sp["pub"] = np.quantile(np.transpose(simList), q  = 0.975, axis = 1)
sp["pmeanSeries"] = np.median(np.transpose(simList), axis = 1)

In [ ]:
#plot graph using ggplot
(ggplot(sp) + 
    geom_line(aes(x = "date", y = "TOTAL")) +
    geom_line(aes(x = "date", y = "pmeanSeries"), color = "blue") +
    geom_ribbon(aes(x = "date", ymin = "plb", ymax = "pub"), alpha = 0.5) +
    scale_x_datetime(breaks = date_breaks('5 days'), labels=date_format('%d-%b')) +
    ggtitle("Series of covid-19 for Sao Paulo") +
    ylab("Number of cases"))